In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('operations').getOrCreate()

In [3]:
df_stock = spark.read.csv(
    '../data/appl_stock.csv',
    inferSchema=True, header=True
)

In [4]:
df_stock.printSchema()
df_stock.show(10)

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            21

In [5]:
df_stock = df_stock.withColumn('Date', df_stock['Date'].cast('timestamp'))
df_stock.printSchema()
df_stock.show(5)

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)

+-------------------+----------+----------+------------------+------------------+---------+------------------+
|               Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:00|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:00|    211.75|212.000006|        209.050005|   

In [6]:
results = df_stock.filter(
    (df_stock['Volume'] > 3e8) & (df_stock['Open'] > df_stock['Close'])
).select(['Open', 'Close', 'Volume']).collect()

In [7]:
results

[Row(Open=205.95000100000001, Close=205.940001, Volume=466777500),
 Row(Open=201.079996, Close=192.060003, Volume=311488100),
 Row(Open=253.830002, Close=246.249989, Volume=321465200),
 Row(Open=243.71001099999998, Close=235.860004, Volume=419004600),
 Row(Open=241.88000899999997, Close=237.75999500000003, Volume=320728800),
 Row(Open=380.440002, Close=373.620007, Volume=301147700),
 Row(Open=374.56998799999997, Close=372.500008, Volume=308419300),
 Row(Open=514.259995, Close=497.669975, Volume=376530000),
 Row(Open=460.000008, Close=450.49997699999994, Volume=365213100),
 Row(Open=451.69001799999995, Close=439.880001, Volume=302006600)]

In [8]:
results[0].asDict()

{'Open': 205.95000100000001, 'Close': 205.940001, 'Volume': 466777500}

In [9]:
df_sales = spark.read.csv(
    '../data/sales_info.csv',
    inferSchema=True, header=True
)

In [10]:
df_sales.printSchema()
df_sales.show()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [11]:
grouped_company = df_sales.groupBy('Company')
grouped_company

In [12]:
grouped_company.mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [13]:
df_sales.agg({'Sales': 'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [14]:
grouped_company.agg({'Sales': 'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [15]:
from pyspark.sql.functions import countDistinct, stddev, avg, format_number

In [16]:
df_sales.select(countDistinct('Company')).collect()

[Row(count(DISTINCT Company)=4)]

In [17]:
df_sales.select(stddev('Sales').alias('std')).select(format_number('std', 2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



In [18]:
df_sales.orderBy(df_sales['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

